In [1]:
# If pytorch-grad-cam isn’t available via pip install,
# you can install directly from GitHub:
# !pip install git+https://github.com/jacobgil/pytorch-grad-cam.git
# Or try the alias:
!pip install grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.1 MB/s eta 0:00:00:00:0100:01
  Created wheel for grad-cam: filename=grad_cam-1.5.5-py3-none-any.whl size=44284 sha256=6d4a4c4fb3e2bc8a3825e9ca7c9b8c8193f3ede9c2aaf1b

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import transforms, models
from PIL import Image
import numpy as np
from pathlib import Path

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

from sklearn.metrics import classification_report, confusion_matrix

In [3]:
# Paths and parameters
DATA_DIR = "/kaggle/input/blood-cell-cancer-all-4class/Blood cell Cancer [ALL]"
BATCH_SIZE = 16
NUM_CLASSES = 4
NUM_EPOCHS = 10
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Class/severity mappings
CLASS_NAMES = ["Benign", "Early Pre-B", "Pre-B", "Pro-B"]
SEVERITY_MSG = {
    0: "A benign hematological entity exhibiting no signs of malignant transformation. The cellular morphology remains consistent with normal hematopoiesis, with preserved nuclear-to-cytoplasmic ratios, orderly chromatin, and the absence of atypical mitotic figures. No immediate clinical concern; routine surveillance may suffice.",
    1: "An early-stage precursor B-cell malignancy, marked by subtle yet significant deviations from normal lymphopoiesis. These cells begin to demonstrate nuclear irregularities, increased nuclear-cytoplasmic ratio, and early chromatin dispersion — a harbinger of uncontrolled proliferation if left unchecked. Clinical intervention is crucial at this incipient stage.",
    2: "A progressed pre-B lymphoblast population displaying clear morphological evidence of malignancy. Nuclear convolutions, prominent nucleoli, and cytoplasmic basophilia are characteristic. The disease at this stage possesses high proliferative potential, posing a significant systemic threat. Prompt and aggressive therapy is often indicated to curtail disease progression.",
    3: "A highly aggressive and immature leukemic state, where pro-B lymphoblasts dominate. These cells exhibit profound anaplasia, scant cytoplasm, and dense chromatin irregularities. Rapid clinical deterioration is a hallmark; immediate, intensive therapeutic strategies are imperative for any hope of remission."
}

# Dataset
class BloodCancerDataset(Dataset):
    def __init__(self, root, transform=None, filter_copy=True):
        self.transform = transform
        self.samples = []
        root = Path(root)
        for sub in root.iterdir():
            if not sub.is_dir(): continue
            name = sub.name
            if 'Benign' in name: label = 0
            elif 'Early Pre-B' in name or 'early Pre-B' in name: label = 1
            elif 'Pro-B' in name and 'Pre-B' not in name: label = 3
            elif 'Pre-B' in name: label = 2
            else: continue
            for img in sub.glob('*.jpg'):
                if filter_copy and 'Copy' in img.name: continue
                self.samples.append((str(img), label))
        if not self.samples:
            raise RuntimeError(f"No images found under {root}")
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert('RGB')
        if self.transform: img = self.transform(img)
        return img, label

# Transforms
tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# Load data
dataset = BloodCancerDataset(DATA_DIR, transform=tfms)
print(f"Total images: {len(dataset)}")
train_n = int(0.8*len(dataset))
val_n = len(dataset) - train_n
train_ds, val_ds = random_split(dataset, [train_n, val_n])
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

Total images: 2847


In [4]:
# Build model function
def build_model():
    m = models.resnet50(pretrained=True)
    m.fc = nn.Linear(m.fc.in_features, NUM_CLASSES)
    return m

# Instantiate model and wrap for multi-GPU
model = build_model().to(DEVICE)
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = nn.DataParallel(model)

# Loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# Training loop with per-class metrics
best_acc = 0.0
save_path = "best_model.pth"
for epoch in range(1, NUM_EPOCHS+1):
    # --- Training ---
    model.train()
    running_loss = correct = total = 0
    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * x.size(0)
        preds = out.argmax(1)
        correct += (preds == y).sum().item()
        total += y.size(0)
    tr_loss = running_loss / total
    tr_acc = correct / total

    # --- Validation ---
    model.eval()
    val_preds, val_targets = [], []
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            out = model(x)
            preds = out.argmax(1)
            val_preds.extend(preds.cpu().tolist())
            val_targets.extend(y.cpu().tolist())
    val_acc = (np.array(val_preds) == np.array(val_targets)).mean()

    # Report per-class metrics
    print(f"Epoch {epoch}/{NUM_EPOCHS} | Train Loss {tr_loss:.4f} | "
          f"Train Acc {tr_acc:.4f} | Val Acc {val_acc:.4f}")
    print(classification_report(val_targets, val_preds, target_names=CLASS_NAMES))
    print("Confusion Matrix:")
    print(confusion_matrix(val_targets, val_preds))

    # Save best
    model_to_save = model.module if isinstance(model, nn.DataParallel) else model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model_to_save.state_dict(), save_path)

    print(f"Epoch {epoch}/{NUM_EPOCHS} | Loss {tr_loss:.4f} | "
          f"Train Acc {tr_acc:.4f} | Val Acc {val_acc:.4f}")

# Load best model weights into a fresh architecture (no DataParallel)
best_model = build_model().to(DEVICE)
best_model.load_state_dict(torch.load(save_path, map_location=DEVICE))
best_model.eval()

# Inference + Grad-CAM
def predict_and_explain(img_path):
    img = Image.open(img_path).convert('RGB')
    inp = tfms(img).unsqueeze(0).to(DEVICE)
    out = best_model(inp)
    cls = out.argmax(1).item()
    print(f"Predicted: {CLASS_NAMES[cls]}")
    print(SEVERITY_MSG[cls])
    cam = GradCAM(model=best_model, target_layers=[best_model.layer4[-1]])
    gcam = cam(input_tensor=inp, targets=[ClassifierOutputTarget(cls)])[0]
    rgb = np.array(img.resize((224,224)), dtype=float) / 255
    vis = show_cam_on_image(rgb, gcam, use_rgb=True)
    Image.fromarray(vis).save("gradcam_out.jpg")
    print("Grad-CAM saved to gradcam_out.jpg")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 188MB/s] 


Using 2 GPUs
Epoch 1/10 | Train Loss 0.2186 | Train Acc 0.9275 | Val Acc 0.9965
              precision    recall  f1-score   support

      Benign       1.00      0.98      0.99        99
 Early Pre-B       0.99      1.00      1.00       142
       Pre-B       0.99      1.00      1.00       163
       Pro-B       1.00      1.00      1.00       166

    accuracy                           1.00       570
   macro avg       1.00      0.99      1.00       570
weighted avg       1.00      1.00      1.00       570

Confusion Matrix:
[[ 97   1   1   0]
 [  0 142   0   0]
 [  0   0 163   0]
 [  0   0   0 166]]
Epoch 1/10 | Loss 0.2186 | Train Acc 0.9275 | Val Acc 0.9965
Epoch 2/10 | Train Loss 0.0728 | Train Acc 0.9785 | Val Acc 0.9982
              precision    recall  f1-score   support

      Benign       1.00      0.99      0.99        99
 Early Pre-B       0.99      1.00      1.00       142
       Pre-B       1.00      1.00      1.00       163
       Pro-B       1.00      1.00      1.00  

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_31/378333631.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more detail

In [5]:

# Example:
predict_and_explain("/kaggle/input/blood-cell-cancer-all-4class/Blood cell Cancer [ALL]/Benign/Sap_013 (10).jpg")

Predicted: Benign
A benign hematological entity exhibiting no signs of malignant transformation. The cellular morphology remains consistent with normal hematopoiesis, with preserved nuclear-to-cytoplasmic ratios, orderly chromatin, and the absence of atypical mitotic figures. No immediate clinical concern; routine surveillance may suffice.
Grad-CAM saved to gradcam_out.jpg
